<img src="assets/parallelisation.png" align="left" width="600" style="margin-right:15px;"/>

In [15]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env
from langchain_anthropic import ChatAnthropic

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

Loaded env from: /Users/alien110/Documents/04_Code/Experiments/ex-agents-patterns/langgraph-essentials/python/.env
OPENAI_API_KEY=****EjUA
ANTHROPIC_API_KEY=****wwAA
LANGSMITH_TRACING_V2=true
LANGSMITH_ENDPOINT=****.com
LANGSMITH_API_KEY=****d3f4
LANGSMITH_PROJECT=****-dev


In [ ]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image

# Graph state
class State(TypedDict):
    network_element: str
    performance_report: str
    security_assessment: str
    capacity_planning: str
    combined_output: str


# Nodes
def call_llm_1(state: State):
    """First LLM call to generate performance report"""

    prompt = f"""Generate a single concise paragraph (4-6 sentences) Network Performance Report for {state['network_element']}.

Include: key KPIs (throughput, latency, availability), current status vs SLA targets, and one key recommendation.

Keep it professional, data-focused, and concise in ONE paragraph only."""

    msg = llm.invoke(prompt)
    return {"performance_report": msg.content}


def call_llm_2(state: State):
    """Second LLM call to generate security assessment"""

    prompt = f"""Generate a single concise paragraph (4-6 sentences) Network Security Assessment for {state['network_element']}.

Include: threat landscape overview, security posture, compliance status, and risk rating with one key recommendation.

Keep it professional, risk-focused, and concise in ONE paragraph only."""

    msg = llm.invoke(prompt)
    return {"security_assessment": msg.content}


def call_llm_3(state: State):
    """Third LLM call to generate capacity planning"""

    prompt = f"""Generate a single concise paragraph (4-6 sentences) Capacity Planning Analysis for {state['network_element']}.

Include: current utilization, growth projection, one key bottleneck, and scaling recommendation.

Keep it professional, forward-looking, and concise in ONE paragraph only."""

    msg = llm.invoke(prompt)
    return {"capacity_planning": msg.content}


def aggregator(state: State):
    """Combine all reports into a single executive dashboard"""

    combined = f"NETWORK INTELLIGENCE DASHBOARD: {state['network_element'].upper()}\n"
    combined += "=" * 80 + "\n\n"
    combined += f"📊 PERFORMANCE REPORT\n{'-' * 80}\n{state['performance_report']}\n\n"
    combined += f"🔒 SECURITY ASSESSMENT\n{'-' * 80}\n{state['security_assessment']}\n\n"
    combined += f"📈 CAPACITY PLANNING\n{'-' * 80}\n{state['capacity_planning']}"
    return {"combined_output": combined}


# Build workflow
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)

# Add edges to connect nodes
parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")
parallel_builder.add_edge("aggregator", END)
parallel_workflow = parallel_builder.compile()

# Show workflow
display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

# Invoke
# state = parallel_workflow.invoke({"network_element": "5G Core Network"})
#print(state["combined_output"])

In [17]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Clear cell output
clear_output(wait=True)

# Create all widgets fresh
topic_input = widgets.Text(
    value='5G Core Network',
    placeholder='Enter network element...',
    description='Network:',
    continuous_update=False,
    layout=widgets.Layout(width='350px')
)

submit_btn = widgets.Button(
    description='Generate Intelligence Report',
    button_style='primary',
    icon='search'
)

result_html = widgets.HTML(value='')

def on_button_click(btn):
    """Handle button click"""
    network_element = topic_input.value.strip()
    
    if not network_element:
        result_html.value = "<p>⚠️ Please enter a network element!</p>"
        return
    
    # Show loading state
    btn.disabled = True
    result_html.value = f"<p>🔄 Analyzing network element: {network_element}...</p>"
    
    try:
        # Invoke workflow
        state = parallel_workflow.invoke({"network_element": network_element})
        
        # Build HTML string with results
        html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 15px; background: #f8f9fa; border-radius: 5px;">
            <h3 style="color: #0066cc; margin-top: 0;">Network Intelligence Dashboard: {network_element}</h3>
            <hr style="border: 1px solid #0066cc;">
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">📊 Performance Report</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #28a745;">{state['performance_report']}</pre>
            </div>
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">🔒 Security Assessment</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #dc3545;">{state['security_assessment']}</pre>
            </div>
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">📈 Capacity Planning</h4>
                <pre style="white-space: pre-wrap; background: white; padding: 10px; border-left: 3px solid #ffc107;">{state['capacity_planning']}</pre>
            </div>
        </div>
        """
        result_html.value = html
        
    except Exception as e:
        result_html.value = f"<p>❌ Error: {str(e)}</p>"
    
    finally:
        btn.disabled = False

# Attach handler
submit_btn.on_click(on_button_click)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🌐 BT Network Intelligence Platform</h3>"),
    widgets.HTML("<p>Enter a network element (e.g., '5G Core Network', 'Fiber Access Network', 'Cloud Edge') and generate parallel intelligence reports</p>"),
    widgets.HBox([topic_input, submit_btn]),
    result_html
]))